# Chapter 12: Hedging #

## Code segment 12.1
US and German rates in Section 12.1

In [3]:
#Code Segment 12.1
import pandas as pd
import qrpm_funcs as qf
import numpy as np
from datetime import datetime
import ssl

#Get exchange rates and US riskfree rates
lastday=qf.LastYearEnd()
seriesnames=['DEXUSEU','DGS1']
while True:
    cdates,ratematrix=qf.GetFREDMatrix(seriesnames,enddate=lastday)
    usd_per_euro=ratematrix[-1][0]
    rt1_us=ratematrix[-1][1]
    if not np.isnan(usd_per_euro): break
    lastday=lastday[:8]+str(int(lastday[-2:])-1)  #Back up over holidays

print("As of:",cdates[-1])
usd_per_euro=ratematrix[-1][0]
rt1_us=ratematrix[-1][1]
print("    Spot dollars per euro (f/d) %.4f" % usd_per_euro)
print("    US 1-year rate rf %.4f" % rt1_us)

#Put end-of-month exchange rates in dataframe for later use
df_fx_d = pd.DataFrame(np.array(ratematrix)[:,0], \
                     index=[datetime.strptime(cd,"%Y-%m-%d") for cd in cdates], \
                     columns=[seriesnames[0]])
#Drop NaNs and save only monthends
df_fx=df_fx_d.dropna().resample('M').last()
#Form log-returns; negative means dollar strengthened
#so it is ln((f/d)1/(f/d)0) where f is dollar d is euro
df_fx=np.log(df_fx).diff().dropna()

#Problem with Bundesbank's certificate - drop verification
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

#Read one-year rates from Bundesbank
bundesbank_url = "https://www.bundesbank.de/statistic-rmi/StatisticDownload?tsId="
bundesbank_url += "BBSIS.M.I.ZST.ZI.EUR.S1311.B.A604.R01XX.R.A.A._Z._Z.A&"
bundesbank_url += "its_csvFormat=en&its_fileFormat=csv&mode=its"
df_csv = pd.read_csv(bundesbank_url,skiprows=4)
df_german = df_csv.rename(index=str, columns={df_csv.columns[0]: "Date", \
                                              df_csv.columns[1]: "Rate"})
#Check latest
bundesbank_lastdate=df_german['Date'].iloc[-1]
if bundesbank_lastdate >= lastday[:7]:
    rt1_german=df_german.loc[df_german['Date'] == lastday[:7]]["Rate"].iloc[0]
    bundesbank_lastdate = lastday[:7]
else: #Bundesbank hasn't yet updated rates to last day of previous year. Mach Schnell!
    rt1_german=df_german["Rate"].iloc[-1]

print("    German 1-year rate rd %.4f" % rt1_german,"as of",bundesbank_lastdate)

#Calculate currency forward price per (12.2)
time=1
fwd_price=usd_per_euro*np.exp((rt1_us-rt1_german)*time/100.)
print("\n    1yr forward dollars per euro %.4f" % fwd_price)

As of: 2021-12-30
    Spot dollars per euro (f/d) 1.1318
    US 1-year rate rf 0.3800


    German 1-year rate rd -0.7300 as of 2021-12

    1yr forward dollars per euro 1.1444


## Code segment 12.2
Table 12.1 in Section 12.1

In [5]:
#Code Segment 12.2
euro_invested=10000.
euro_forward=euro_invested*np.exp(rt1_german/100*time)
usd_invested=euro_invested*usd_per_euro
usd_forward=usd_invested*np.exp(rt1_us/100*time)

def show_currency_forward_table(title):

    print(title)
    print("Item      |        Dollars (USD=f)       |       Euros (EUR=d)     |        Ratio")
    print("-"*90)
    print("Time 0    |", \
         "          {:8,.1f}".format(usd_invested), \
         "          |        {:8,.1f}".format(euro_invested), \
          "        | Spot USD/EUR {:6,.4f}".format(usd_per_euro))
    print("1yr rfree |", \
         "          {:8,.3f}".format(rt1_us), \
         "          |        {:8,.3f}".format(rt1_german), \
          "        |")
    print("Time 1    |", \
         "          {:8,.1f}".format(usd_forward), \
         "          |        {:8,.1f}".format(euro_forward), \
          "        |  Fwd USD/EUR {:6,.4f}".format(fwd_price))

show_currency_forward_table("Table 12.1: Hedging EUR 10,000")

Table 12.1: Hedging EUR 10,000
Item      |        Dollars (USD=f)       |       Euros (EUR=d)     |        Ratio
------------------------------------------------------------------------------------------
Time 0    |           11,318.0           |        10,000.0         | Spot USD/EUR 1.1318
1yr rfree |              0.380           |          -0.730         |
Time 1    |           11,361.1           |         9,927.3         |  Fwd USD/EUR 1.1444


## Code segment 12.3
Table 12.2 in Section 12.1

In [6]:
#Code Segment 12.3
show_currency_forward_table("Table 12.2: Hedging plus gain")

#Calculations for Philippe's investment in TSLA
#under dollar-crash/TSLA up scenario
underlying_dollar_appreciation=.2
new_usd_per_euro=5
new_tsla_value=usd_invested*(1+underlying_dollar_appreciation)
unhedged_tsla_gain_usd=new_tsla_value-usd_forward
unhedged_tsla_gain_euro=unhedged_tsla_gain_usd/new_usd_per_euro
final_euro_total=euro_forward+unhedged_tsla_gain_euro

print("T=1 TSLA  |", \
      "          {:8,.1f}".format(new_tsla_value), \
      "          |                        ", \
      "|              {:6,.4f}".format(underlying_dollar_appreciation))
print("T=1 Unhdgd|", \
     "          {:8,.1f}".format(unhedged_tsla_gain_usd), \
     "          |        {:8,.1f}".format(unhedged_tsla_gain_euro), \
     "        |Crash USD/EUR {:6,.4f}".format(new_usd_per_euro))
print("Final amt |", \
     "                  ", \
     "          |        {:8,.1f}".format(final_euro_total), \
     "        | Final return {:6,.4f}".format(final_euro_total/euro_invested-1.))
print("No hedge  |", \
     "          {:8,.1f}".format(new_tsla_value), \
     "          |        {:8,.1f}".format(new_tsla_value/new_usd_per_euro), \
     "        |Final no hdg {:6,.4f}".format( \
                        (new_tsla_value/new_usd_per_euro)/euro_invested-1.))


Table 12.2: Hedging plus gain
Item      |        Dollars (USD=f)       |       Euros (EUR=d)     |        Ratio
------------------------------------------------------------------------------------------
Time 0    |           11,318.0           |        10,000.0         | Spot USD/EUR 1.1318
1yr rfree |              0.380           |          -0.730         |
Time 1    |           11,361.1           |         9,927.3         |  Fwd USD/EUR 1.1444
T=1 TSLA  |           13,581.6           |                         |              0.2000
T=1 Unhdgd|            2,220.5           |           444.1         |Crash USD/EUR 5.0000
Final amt |                              |        10,371.4         | Final return 0.0371
No hedge  |           13,581.6           |         2,716.3         |Final no hdg -0.7284


## Code segment 12.4
Quanto hedging inputs in Section 12.1.1

In [7]:
#Code Segment 12.4
#Parameter calculations for dollar/euro TSLA quanto
import yfinance as yf
#https://github.com/ranaroussi/yfinance

tsla = yf.Ticker("TSLA")
df_tsla=tsla.history(period="max")   #Prices are adjusted
df_tsla=df_tsla.resample('M').last()
#Dumb down dates
dumb_index=[]; close=[]
for i in range(len(df_tsla)):
    dumb_index.append(df_tsla.index[i].to_pydatetime().replace(tzinfo=None))
    close.append(df_tsla["Close"].iloc[i])
    
df_tsla=pd.DataFrame(index=dumb_index,data=close)
df_tsla.index.rename("Date", inplace=True)
df_tsla.columns = ['Close']

#Form log-returns
df_tsla=pd.DataFrame(np.log(df_tsla["Close"]).diff().dropna())

# Join underlying and currency
df_join=df_tsla.join(df_fx).dropna()

print("Data from",df_join.index[0].strftime("%Y-%m-%d"),"to", \
      df_join.index[-1].strftime("%Y-%m-%d"))
#Current price of underlying in foreign currency
X_0=usd_invested
print("            Current price of underlying X_0: %.2f" % X_0)
print("                             Time horizon T: %f" % time)
print("                 Domestic riskfree rate r_d: %.4f" % rt1_german)
print("                  Foreign riskfree rate r_f: %.4f" % rt1_us)
print("          Foreign currency strike price K_f: %.2f" % usd_forward)

#Annualized standard deviations
sigma_X=df_join.std()["Close"]*np.sqrt(12.)
sigma_R=df_join.std()[df_fx.columns[0]]*np.sqrt(12.)
print("Annualized volatility of underlying sigma_X: %.4f" % sigma_X)
print("  Annualized volatility of f/x rate sigma_R: %.4f" % sigma_R)
#Correlation
rho=df_join.corr().iloc[0,1]
print("     Correlation between underlying and f/x: %.4f" % rho)

Data from 2010-07-31 to 2021-12-31
            Current price of underlying X_0: 11318.00
                             Time horizon T: 1.000000
                 Domestic riskfree rate r_d: -0.7300
                  Foreign riskfree rate r_f: 0.3800
          Foreign currency strike price K_f: 11361.09
Annualized volatility of underlying sigma_X: 0.5562
  Annualized volatility of f/x rate sigma_R: 0.0843
     Correlation between underlying and f/x: 0.1512


## Code segment 12.5
Quanto hedging outputs in Section 12.1.1

In [8]:
#Code Segment 12.5
import scipy.stats as spst

#Prices of quanto call and quanto put
def quanto_call(X_0,T,r_f,r_d,K_f,sigma_X,sigma_R,rho):
    #returns price of quanto call from (12.23)
    #Variables have to be positive
    if X_0<=0 or T<=0 or K_f<=0 or sigma_X<0 or sigma_R<0:
        return(0)
    q=r_d-r_f-rho*sigma_X*sigma_R
    d_plus=(np.log(X_0/K_f)+(r_d-q+sigma_X**2/2)*T)/(sigma_X*np.sqrt(T))
    d_minus=d_plus-sigma_X*np.sqrt(T)
    #Get cumulative normals
    n_of_d_plus=spst.norm.cdf(d_plus)
    n_of_d_minus=spst.norm.cdf(d_minus)
    qc=X_0*np.exp(-q*T)*n_of_d_plus-K_f*np.exp(-r_d*T)*n_of_d_minus
    return(qc)

def quanto_put(X_0,T,r_f,r_d,K_f,sigma_X,sigma_R,rho):
    #compute price of quanto put from (12.25)
    #calls quanto_call and uses put-call parity
    qc=quanto_call(X_0,T,r_f,r_d,K_f,sigma_X,sigma_R,rho)
    qp=qc-X_0+np.exp(-r_d*T)*K_f
    
    return(qp)

#Compute example quanto call value
q_c = quanto_call(X_0, time, rt1_us/100., rt1_german/100., usd_forward, \
            sigma_X, sigma_R, rho)
#Scale by 1/fwd
q_c/=fwd_price
print("              Buy Quanto call: %.2f" % q_c)

#Compute example plain vanilla call value
bs_c = quanto_call(X_0, time, rt1_us/100., rt1_us/100., usd_forward, \
            sigma_X, 0., 0.)
print("       Write Dollar-only call: %.2f" % bs_c)

up_cost=q_c-bs_c
print("  Cost of hedging TSLA upside: %.2f" % up_cost)

#Compute example quanto put value
q_p = quanto_put(X_0, time, rt1_us/100., rt1_german/100., usd_forward, \
            sigma_X, sigma_R, rho)
#Scale by 1/fwd
q_p/=fwd_price
print("\n             Write Quanto put: %.2f" % q_p)

#Compute example plain vanilla put value
bs_p = quanto_put(X_0, time, rt1_us/100., rt1_us/100., usd_forward, \
            sigma_X, 0., 0.)
print("          Buy Dollar-only put: %.2f" % bs_p)

down_cost=bs_p-q_p
print("Cost of hedging TSLA downside: %.2f "% down_cost)
total_cost=up_cost+down_cost
print("\n        Total cost of hedging: %.2f" % total_cost)

              Buy Quanto call: 2234.26
       Write Dollar-only call: 2479.48
  Cost of hedging TSLA upside: -245.21

             Write Quanto put: 2344.65
          Buy Dollar-only put: 2479.48
Cost of hedging TSLA downside: 134.83 

        Total cost of hedging: -110.39


## Code segment 12.6
Table 12.3 in Section 12.2

In [9]:
#Code Segment 12.6
pwc_french_euro_budget=20000000.
french_bottle_cost=10.
us_bottle_cost=french_bottle_cost*usd_per_euro
french_bottle_sales_price=french_bottle_cost*1.1
pwc_num_imported=pwc_french_euro_budget/french_bottle_cost

print("Table 12.3: Philippe Wine Company Summary")
print("Location     |       France")
print("CEO          |       Philippe")
print("Import budget| EUR {:12,.1f}".format(pwc_french_euro_budget))
print("Cost/bottle  |    USD {:7.4f}".format(us_bottle_cost))
print("Sales price  |    EUR {:7.4f}".format(french_bottle_sales_price))
print("Num imported |     {:12,.1f}".format(pwc_num_imported))

print("\nT=0 USD/EUR  |    USD {:7.4f}".format(usd_per_euro))
print("Cost/bot, lcl|    EUR {:7.4f}".format(french_bottle_cost))
print("Profit       |    EUR {:12,.1f}". \
      format(pwc_num_imported*(french_bottle_sales_price-french_bottle_cost)))

fx_bad=us_bottle_cost/french_bottle_sales_price
print("\nBad USD/EUR  |    USD {:7.4f}".format(fx_bad))
print("Cost/bot, lcl|    EUR {:7.4f}".format(us_bottle_cost/fx_bad))
print("Profit       |    EUR {:12,.1f}".format(0))

Table 12.3: Philippe Wine Company Summary
Location     |       France
CEO          |       Philippe
Import budget| EUR 20,000,000.0
Cost/bottle  |    USD 11.3180
Sales price  |    EUR 11.0000
Num imported |      2,000,000.0

T=0 USD/EUR  |    USD  1.1318
Cost/bot, lcl|    EUR 10.0000
Profit       |    EUR  2,000,000.0

Bad USD/EUR  |    USD  1.0289
Cost/bot, lcl|    EUR 11.0000
Profit       |    EUR          0.0


## Code segment 12.7
Table 12.4 in Section 12.2

In [10]:
#Code Segment 12.7
#Calculations for Philippe-Ken Wine Company

def display_company_stats(new_rate,title):

    french_euro_budget=10000000.
    french_bottle_cost=10.
    us_bottle_cost=french_bottle_cost*usd_per_euro
    us_usd_budget=french_euro_budget*usd_per_euro
    french_bottle_sales_price=french_bottle_cost*1.1
    us_bottle_sales_price=us_bottle_cost*1.1

    print(title)
    print("Location     |       France      |       USA        |    Total")
    print("CEO          |       Philippe    |       Ken        |")
    print("Import budget| EUR {:12,.1f}".format(french_euro_budget), \
          " | USD {:12,.1f}".format(us_usd_budget),"|")
    print("Cost/bottle  |    USD {:7.4f}".format(us_bottle_cost), \
          "   |    EUR {:7.4f}".format(french_bottle_cost),"  |")
    print("Sales price  |    EUR %.4f" % french_bottle_sales_price, \
      "   |    USD %.4f" % us_bottle_sales_price,"  |")

    print("\n")
    local_french_cost=us_bottle_cost/new_rate
    local_us_cost=french_bottle_cost*new_rate
    french_bottles=french_euro_budget*new_rate/us_bottle_cost
    us_bottles=us_usd_budget/new_rate/french_bottle_cost
    total_bottles=french_bottles+us_bottles
    french_pandl=french_bottles*(11-us_bottle_cost/new_rate)
    us_pandl=us_bottles*(us_bottle_sales_price-french_bottle_cost*new_rate)
    total_pandl=french_pandl+us_pandl/new_rate

    print("USD/EUR      |    %.4f" % new_rate,"        |                  |")

    print("Cost/bot lcl |    EUR {:7.4f}".format(local_french_cost), \
          "   |    USD {:7.4f}".format(local_us_cost),"  |")
    print("Num imported |     {:12,.1f}".format(french_bottles), \
      " |     {:12,.1f}".format(us_bottles), \
      "|    {:12,.1f}".format(total_bottles))
    print("Profit       |  EUR{:12,.1f}".format(french_pandl), \
      " |  USD{:12,.1f}".format(us_pandl), \
      "| EUR{:12,.1f}".format(total_pandl))

#Show table at current exchange rate
display_company_stats(usd_per_euro, \
                "Table 12.4: Philippe-Ken Wine Company: unchanged f/x")

Table 12.4: Philippe-Ken Wine Company: unchanged f/x
Location     |       France      |       USA        |    Total
CEO          |       Philippe    |       Ken        |
Import budget| EUR 10,000,000.0  | USD 11,318,000.0 |
Cost/bottle  |    USD 11.3180    |    EUR 10.0000   |
Sales price  |    EUR 11.0000    |    USD 12.4498   |


USD/EUR      |    1.1318         |                  |
Cost/bot lcl |    EUR 10.0000    |    USD 11.3180   |
Num imported |      1,000,000.0  |      1,000,000.0 |     2,000,000.0
Profit       |  EUR 1,000,000.0  |  USD 1,131,800.0 | EUR 2,000,000.0


## Code segment 12.8
Table 12.5 in Section 12.2

In [11]:
#Code Segment 12.8
display_company_stats(1.0,"Table 12.5: Philippe-Ken Wine Company: strong dollar")

Table 12.5: Philippe-Ken Wine Company: strong dollar
Location     |       France      |       USA        |    Total
CEO          |       Philippe    |       Ken        |
Import budget| EUR 10,000,000.0  | USD 11,318,000.0 |
Cost/bottle  |    USD 11.3180    |    EUR 10.0000   |
Sales price  |    EUR 11.0000    |    USD 12.4498   |


USD/EUR      |    1.0000         |                  |
Cost/bot lcl |    EUR 11.3180    |    USD 10.0000   |
Num imported |        883,548.3  |      1,131,800.0 |     2,015,348.3
Profit       |  EUR  -280,968.4  |  USD 2,772,683.6 | EUR 2,491,715.3


## Code segment 12.9
Table 12.6 in Section 12.2

In [12]:
#Code Segment 12.9
display_company_stats(1.5,"Table 12.6: Philippe-Ken Wine Company: weak dollar")

Table 12.6: Philippe-Ken Wine Company: weak dollar
Location     |       France      |       USA        |    Total
CEO          |       Philippe    |       Ken        |
Import budget| EUR 10,000,000.0  | USD 11,318,000.0 |
Cost/bottle  |    USD 11.3180    |    EUR 10.0000   |
Sales price  |    EUR 11.0000    |    USD 12.4498   |


USD/EUR      |    1.5000         |                  |
Cost/bot lcl |    EUR  7.5453    |    USD 15.0000   |
Num imported |      1,325,322.5  |        754,533.3 |     2,079,855.8
Profit       |  EUR 4,578,547.4  |  USD-1,924,210.9 | EUR 3,295,740.2


## Code segment 12.10
Figure 12.1 in Section 12.2.1

In [13]:
#Code Segment 12.10
#Show number-of-bottles curve for Siegel's paradox
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']= 300

x=np.arange(usd_per_euro-.3,usd_per_euro+.3+.01,.01)
z=x/usd_per_euro
y=z+1/z

i=np.arange(len(x))
plt.plot(i,y,color='red')
plt.xticks(i[::10],[str(round(float(label), 3)) for label in x[::10]])
plt.xlabel("Dollars per Euro")
plt.ylabel("Millions of bottles")
plt.title("Figure 12.1: Total bottle volume of Philippe-Ken Wine Company")
plt.grid()
plt.show()

## Code segment 12.11
Siegel's Paradox calculations in Section 12.2.1

In [14]:
#Code Segment 12.11
#More Siegel's Paradox calculations
#Roots of quadratic in (12.12)
a_poly=11
b_poly=-1
c_poly=-11
discriminant=b_poly**2-4*a_poly*c_poly

low_root=(-b_poly-np.sqrt(discriminant))/(2*a_poly)
high_root=(-b_poly+np.sqrt(discriminant))/(2*a_poly)
print("Roots of (12.12) polynomial:",low_root,1,high_root)

print("Profit less than original between (f/d)=",usd_per_euro, \
      "and",high_root*usd_per_euro)

#minimum profit location
min_root=(4+np.sqrt(158/3))/11
print("Minimum profit at z=",min_root,"(f/d)=",min_root*usd_per_euro)

#minimum profit amount
min_profit=1000000*(min_root+1)*(11-21*min_root+11*min_root**2)/min_root**2
print("Minimum hedged profit",min_profit)
profit_giveup_percent=100.-min_profit/20000.
print("Percentage profit giveup at minimum: %.2f" % profit_giveup_percent)
#Compute unhedged profit at that point
print("Unhedged (PWC) profit at minimum hedged profit:",20000000*(1.1*min_root-1))

Roots of (12.12) polynomial: -0.9555779793429654 1 1.0464870702520563
Profit less than original between (f/d)= 1.1318 and 1.1844140661112772
Minimum profit at z= 1.0233800320326436 (f/d)= 1.1582615202545459
Minimum hedged profit 1988770.9045979201
Percentage profit giveup at minimum: 0.56
Unhedged (PWC) profit at minimum hedged profit: 2514360.7047181595


## Code segment 12.12
Figures 12.2 and 12.3 in Section 12.2.1

In [15]:
#Code Segment 12.12
#Show profitability graphs for Siegel's paradox -
#Philippe-Ken Wine Company

#Broad range and narrow range
x=np.arange(.5,2.01,.01)
title_str="Figure 12.2: Philippe-Ken Wine Company Profit"

for graph_no in [1,2]:

    z=x/usd_per_euro
    y_hedged=[(zz+1)*(11-21*zz+11*zz**2)/zz**2 for zz in z]  #(12.29)
    y_unhedged=20*(1.1*z-1)
    y_constant=[2]*len(y_hedged)

    i=np.arange(len(x))
    plt.plot(i,y_hedged,color='red',label='US/France PKWC')
    if graph_no==1:
        plt.plot(i,y_unhedged,color='blue',label='France only PWC')
    plt.plot(i,y_constant,color='green',label='Unchanged rates')
    plt.xticks(i[::20],[str(round(float(label), 3)) for label in x[::20]])
    plt.xlabel("Dollars per Euro")
    plt.ylabel("Millions of Euros")
    plt.legend(loc='lower right')
    plt.title(title_str)
    plt.grid()
    plt.show()

    #next iteration
    x=np.arange(usd_per_euro,high_root*usd_per_euro+.001,.001)
    title_str="Figure 12.3: Philippe-Ken Wine Company Profit - neighborhood"

## Code segment 12.13
Figure 12.4 in Section 12.2.4

In [16]:
#Code Segment 12.13
#Get Treasury rates; compare them to SOFR rates from IHS Markit
import pandas as pd
import numpy as np
import qrpm_funcs as qf
import matplotlib.pyplot as plt
%matplotlib inline

fname="USD SOFR"
df_out = pd.read_excel(fname+".xlsx",engine="openpyxl")
#Drop unused old index column
df_out = df_out.drop(df_out.columns[0], axis=1)

seriesnames = ['DGS3MO','DGS1','DGS5','DGS10','DGS20','DGS30']
tenors = [3.,12.,60.,120.,240.,360.]
firstday = df_out["Reference Date"].iloc[0].strftime('%Y-%m-%d')
lastday = qf.LastYearEnd()

cdates,ratematrix = qf.GetFREDMatrix(seriesnames,startdate=firstday, \
                        enddate=lastday)

df_tsys = pd.DataFrame(ratematrix,columns=seriesnames)
df_tsys["Date"] = cdates

df_tsys.dropna(inplace=True)    #Drop NaNs
df_tsys.set_index("Date",inplace=True)    #Make the date the index

#Combine Treasurys and SOFR so spreads can be computed
df_both = df_tsys.copy()
for t,m in enumerate(tenors):
    #Form dataframe with SOFR rates
    df_m = pd.DataFrame(list(df_out["Interest Rate"] \
                    [df_out["Int Months"]==m]), \
                    columns=["X"+seriesnames[t]])
    df_m["Date"] = list(df_out["Reference Date"] \
                    [df_out["Int Months"]==m].dt.strftime('%Y-%m-%d'))
    df_m.set_index("Date",inplace=True)
    df_both = df_both.join(df_m, how = 'inner')

#Form spread series
for series in seriesnames:
    df_both["S"+series[3:]]=df_both["X"+series]-df_both[series]
    #Drop components
    df_both = df_both.drop(columns=["X"+series,series])

figtitle="Figure 12.4: US swap spreads over Treasurys"+ \
    "(by maturity)\nSource:IHS Markit"
df_both.plot(title=figtitle,ylabel="pct",grid=True,rot=45)

<AxesSubplot: title={'center': 'Figure 12.4: US swap spreads over Treasurys(by maturity)\nSource:IHS Markit'}, xlabel='Date', ylabel='pct'>

## Code segment 12.14
Swap spread profits in Section 12.2.4

In [17]:
#Code Segment 12.14
spread_profit = - 10**8 * df_both["S30"].iloc[-1] * 10**(-2)
print("Profit from $100Mn 30-year spread: ${:,}". \
      format(spread_profit),"as of",df_both.index[-1])

Profit from $100Mn 30-year spread: $439,330.0 as of 2021-12-31


## Code segment 12.15
Figures 12.6a and 12.6b in Section 12.4.1

In [18]:
#Show put buying

#Graphing code used in all subsequent distribution shaping figures
def dual_dist_hedge_graph(x,y,y_norm,leg_trade,title1,title2):
    plt.figure(figsize=(12,4))
    #Payoff graph
    plt.subplot(1,2,1)
    plt.plot(x,x,label='Underlying')
    plt.plot(x,y,label=leg_trade)
    plt.xlabel("Underlying at maturity")
    plt.ylabel("Value at maturity")
    plt.grid()
    plt.legend()
    plt.title(title1)
    #Distribution graph
    plt.subplot(1,2,2)
    plt.xlabel("Underlying")
    plt.ylabel("Probability density")
    plt.plot(x,y_norm,label='Original')
    plt.plot(y,y_norm,label=leg_trade)
    plt.grid()
    plt.legend()
    plt.title(title2)
    
    plt.show()

#Front ends for quanto_call and quanto_put routines above
#Simplifies them to Black-Scholes (same as formula 9.3)
def black_scholes_call(X,K,T,r,sigma):
    return(quanto_call(X,T,r,r,K,sigma,0.,1.))
def black_scholes_put(X,K,T,r,sigma):
    return(quanto_put(X,T,r,r,K,sigma,0.,1.))
    
#Common parameters used in all subsequent distribution shaping figures
money=100
time=1
rfree=.02
sigma=.2
x=np.arange(50.,150.,1.)
y_norm=spst.norm.pdf(np.log(x),np.log(money)+rfree*time,sigma)

#Buy-put trade
put_strike=90
put_cost=black_scholes_put(money,put_strike,time,rfree,sigma)

under_with_trade=[max(put_strike-put_cost,u-put_cost) for u in x]
title_1="Figure 12.6a: Payoff of Put plus Underlying:\nStrike=%3.f" % put_strike
title_1+=", Cost=%2.2f" % put_cost
title_2="Figure 12.6b: Density function with and w/o put"
legend="W/put"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.16
Figures 12.7a and 12.7b in Section 12.4.1

In [19]:
#Code Segment 12.16
#Show selling call

call_strike=110
call_cost=black_scholes_call(money,call_strike,time,rfree,sigma)

under_with_trade=[min(call_strike+call_cost,u+call_cost) for u in x]
title_1="Figure 12.7a: Payoff of Underlying minus Call:\nStrike=%3.f" % call_strike
title_1+=", Cost=%2.2f" % call_cost
title_2="Figure 12.7b: Density function with and w/o written call"
legend="W/call"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.17
Figures 12.8a and 12.8b in Section 12.4.1

In [20]:
#Code Segment 12.17
#Show put spread

low_put_strike=80
high_put_strike=90
low_put_cost=black_scholes_put(money,low_put_strike,time,rfree,sigma)
high_put_cost=black_scholes_put(money,high_put_strike,time,rfree,sigma)

under_with_trade=np.concatenate(( \
    [u+high_put_strike-low_put_strike for u in x if u<=low_put_strike], \
    [high_put_strike for u in x if low_put_strike<u<high_put_strike], \
    [u for u in x if u>=high_put_strike]))
under_with_trade-=(high_put_cost-low_put_cost)

title_1="Figure 12.8a: Payoff of Put Spread plus Underlying:\nHighStrk=%3.f" \
    % high_put_strike
title_1+=", HighCst=%2.2f" % high_put_cost
title_1+=";\nLowStrk=%3.f" % low_put_strike
title_1+=", LowCst=%2.2f" %low_put_cost
title_2="Figure 12.8b: Density function with and w/o put spread"
legend="W/put spread"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.18
Figures 12.9a and 12.9b in Section 12.4.1

In [21]:
#Code Segment 12.18
#Show (bear) call spread

low_call_strike=110
high_call_strike=120
low_call_cost=black_scholes_call(money,low_call_strike,time,rfree,sigma)
high_call_cost=black_scholes_call(money,high_call_strike,time,rfree,sigma)

under_with_trade=np.concatenate(( \
    [u for u in x if u<=low_call_strike], \
    [low_call_strike for u in x if low_call_strike<u<high_call_strike], \
    [u+low_call_strike-high_call_strike for u in x if u>=high_call_strike]))
under_with_trade+=(low_call_cost-high_call_cost)

title_1="Figure 12.9a: Payoff of Call Spread plus Underlying:\nHighStrk=%3.f" \
    % high_call_strike
title_1+=", HighCst=%2.2f" % high_call_cost
title_1+=";\nLowStrk=%3.f" % low_call_strike
title_1+=", LowCst=%2.2f" %low_call_cost
title_2="Figure 12.9b: Density function with and w/o call spread"
legend="W/call spread"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.19
Figures 12.10a and 12.10b in Section 12.4.1

In [22]:
#Code Segment 12.19
#Show a long straddle

call_cost=black_scholes_call(money,money,time,rfree,sigma)
put_cost=black_scholes_put(money,money,time,rfree,sigma)

under_with_trade=np.concatenate(\
    ([money-call_cost-put_cost for u in x if u<=money], \
    [u+(u-money)-call_cost-put_cost for u in x if u>money]))

title_1="Figure 12.10a: Payoff of Underlying Plus Straddle:"+ \
    "\nCall Strike=Put Strike=%3.f" % money
title_1+=",\nCall Cost=%2.2f" % call_cost
title_1+=", Put Cost=%2.2f" % put_cost
title_2="Figure 12.10b: Density function with and w/o +straddle"
legend="W/+straddle"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.20
Figures 12.11a and 12.11b in Section 12.4.1

In [23]:
#Code Segment 12.20
#Show a short straddle

under_with_trade=np.concatenate(\
    ([u+(u-money)+call_cost+put_cost for u in x if u<=money], \
    [money+call_cost+put_cost for u in x if u>money]))

title_1="Figure 12.11a: Payoff of Underlying Minus Straddle:"+ \
    "\nCall Strike=Put Strike=%3.f" % money
title_1+=",\nCall Cost=%2.2f" % call_cost
title_1+=", Put Cost=%2.2f" % put_cost
title_2="Figure 12.11b: Density function with and w/o -straddle"
legend="W/-straddle"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.21  
Figures 12.12a and 12.12b in Section 12.4.1

In [24]:
#Code Segment 12.21
#Show a (short) strangle

put_strike=90
call_strike=110

put_cost=black_scholes_put(money,put_strike,time,rfree,sigma)
call_cost=black_scholes_call(money,call_strike,time,rfree,sigma)

under_with_trade=np.concatenate(\
    ([u+(u-put_strike) for u in x if u<=put_strike], \
     [u for u in x if put_strike<u<call_strike], \
     [call_strike for u in x if u>=call_strike]))
under_with_trade+=call_cost+put_cost

title_1="Figure 12.12a: Payoff of Underlying with Strangle:\nCall Strike=%3.f" \
    % call_strike
title_1+=", Call Cost=%2.2f" % call_cost
title_1+=";\nPut Strike=%3.f" % put_strike
title_1+=", Put Cost=%2.2f" % put_cost
title_2="Figure 12.12b: Density function with and w/o strangle"
legend="W/strangle"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.22
Figures 12.13a and 12.13b in Section 12.4.1

In [25]:
#Code Segment 12.22
#Show payoff pattern of a butterfly spread

low_strike=80
mid_strike=100
high_strike=120

low_call_cost=black_scholes_call(money,low_strike,time,rfree,sigma)
mid_call_cost=black_scholes_call(money,mid_strike,time,rfree,sigma)
high_call_cost=black_scholes_call(money,high_strike,time,rfree,sigma)

under_with_trade=np.concatenate(\
    ([u for u in x if u<=low_strike], \
     [u+(u-low_strike) for u in x if low_strike<u<=mid_strike], \
     [u+ (high_strike-u)*(mid_strike-low_strike)/(high_strike-mid_strike) \
      for u in x if mid_strike<u<=high_strike], \
     [u for u in x if u>high_strike]))
trade_cost=low_call_cost-2*mid_call_cost+high_call_cost
under_with_trade-=trade_cost

title_1="Figure 12.13a: Payoff of Underlying w/Butterfly Spread:\nLow=%3.f" \
    % low_strike
title_1+="; Mid=%3.f" % mid_strike
title_1+="; High=%3.f" % high_strike
title_1+="; Trade cost=%2.2f" % trade_cost
title_2="Figure 12.13b: Density function w/ and w/o butterfly"
legend="W/butterfly"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.23
Figures 12.14a and 12.14b in Section 12.4.1

In [26]:
#Code Segment 12.23
#Show payoff pattern of a zero cost collar

put_strike=90
call_strike=117.50468   #just did trial and error to find strike that matches prices

put_cost=black_scholes_put(money,put_strike,time,rfree,sigma)
call_cost=black_scholes_call(money,call_strike,time,rfree,sigma)

under_with_trade=np.concatenate(\
    ([put_strike for u in x if u<=put_strike], \
     [u for u in x if put_strike<u<=call_strike], \
     [call_strike for u in x if call_strike<u]))

title_1="Figure 12.14a: Payoff of Underlying with ZC Collar:\nPut Strike=%3.f" \
    % put_strike
title_1+="; Call Strike=%3.f" % call_strike
title_1+="; Option costs=%2.2f" % put_cost
title_2="Figure 12.14b: Density function with and w/o ZC collar"
legend="W/ZC collar"

dual_dist_hedge_graph(x,under_with_trade,y_norm,legend,title_1,title_2)

## Code segment 12.24  
Simple delta example in Section 12.4.2

In [27]:
#Code Segment 12.24
#Simple delta example

def greeks(price,strike,r,time,sigma):
    #return a 6-vector of Black-Scholes Greeks:
    #BS price, delta, gamma, theta, vega, rho
    if price<=0 or strike<=0 or time<=0:
        return(6*[0])
    #Black-Scholes price
    d1=(np.log(price/strike)+(r+sigma**2/2)*time)/(sigma*np.sqrt(time))
    d2=d1-sigma*np.sqrt(time)
    n_of_d1=spst.norm.cdf(d1)
    n_of_d2=spst.norm.cdf(d2)
    bs_call=price*n_of_d1-np.exp(-r*time)*strike*n_of_d2
    #Greeks
    delta=n_of_d1
    gamma=np.exp(-d1**2/2)/(price*sigma*np.sqrt(2*np.pi*time))
    vega=time*gamma*sigma*(price**2)
    rho=strike*n_of_d2*time*np.exp(-r*time)
    theta=.5*((price*sigma)**2)*gamma+r*rho/time
    gr_vec=[bs_call,delta,gamma,theta,vega,rho]
    return(gr_vec)

sigma=.2
rf=.02
X0=K=100
T=.25

Orig_price,delta,gamma,theta,vega,rho=greeks(X0,K,rf,T,sigma)

X0_new=100.1
New_price,new_d,new_g,new_t,new_v,new_r=greeks(X0_new,K,rf,T,sigma)
diff_delta=(New_price-Orig_price)/(X0_new-X0)

print("     Original X0: %.2f" % X0)
print("  Original Price: %.4f" % Orig_price)
print("           Delta: %.4f" % delta)
print("\n          New X0: %.2f" % X0_new)
print("       New Price: %.4f" % New_price)
print("Difference delta: %.4f" % diff_delta)

     Original X0: 100.00
  Original Price: 4.2322
           Delta: 0.5398

          New X0: 100.10
       New Price: 4.2863
Difference delta: 0.5418


## Code segment 12.25
Figure 12.14 in Section 12.4.2

In [28]:
#Code Segment 12.25
#Graph theta from previous example as a function of strike

strikes=np.arange(50,151,1)

plt.plot(strikes,[greeks(X0,k,rf,T,sigma)[3] \
                 for k in strikes],label="Call")
plt.plot(strikes,[greeks(X0,k,rf,T,sigma)[3]+rf*k*np.exp(-rf*T) \
                 for k in strikes],label="Put")
plt.xlabel("Strike price")
plt.ylabel("Theta")
plt.legend()
plt.grid()
str_title="Figure 12.14: Theta as a function of strike price\n"
str_title+="Price=%3.f" % X0
str_title+=", r=%3.2f" % rf
str_title+=", T=%3.2f" % T
str_title+=", sigma=%3.2f" % sigma
plt.title(str_title)
plt.show();

print("ATM call theta",greeks(X0,X0,rf,T,sigma)[3])
print("ATM put theta",greeks(X0,X0,rf,T,sigma)[3]+rf*X0*np.exp(-rf*T))

ATM call theta 8.93406342873292
ATM put theta 10.924088387118283


## Code segment 12.26
Figure 12.16 in Section 12.4.2

In [29]:
#Code Segment 12.26
#Gamma and vega

fig=plt.figure() # Create matplotlib figure

#Set up two axes because of different orders of
#magnitude between gamma and vega
ax = fig.add_subplot(111)
ax2 = ax.twinx() # Create another axis that shares the same x-axis as ax.

ax.plot(strikes,[greeks(X0,k,rf,T,sigma)[2] \
                 for k in strikes], \
                 color='black')
ax2.plot(strikes,[greeks(X0,k,rf,T,sigma)[4] \
                 for k in strikes], label="Vega", \
                 color='black')

ax.set_ylabel('Gamma', color='blue')
ax.yaxis.label.set_color('blue')
ax.tick_params(axis='y', colors='blue')

ax2.set_ylabel('Vega', color='red')
ax2.yaxis.label.set_color('red')
ax2.tick_params(axis='y', colors='red')

ax.set_xlabel("Strike price")
ax.grid(axis="x")
plt.grid()

str_title="Figure 12.16: Gamma,Vega as a function of strike price\n"
str_title+="Price=%3.f" % X0
str_title+=", r=%3.2f" % rf
str_title+=", T=%3.2f" % T
str_title+=", sigma=%3.2f" % sigma
plt.title(str_title)
plt.show();

print("ATM delta",greeks(X0,X0,rf,T,sigma)[1])
print("ATM gamma ",greeks(X0,X0,rf,T,sigma)[2])
print("ATM vega ",greeks(X0,X0,rf,T,sigma)[4])
print("Vega/gamma ratio ",X0**2*T*sigma)

ATM delta 0.539827837277029
ATM gamma  0.039695254747701185
ATM vega  19.847627373850596
Vega/gamma ratio  500.0
